In [1]:
pip install prophet

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pyspark.sql.functions as F
from prophet import Prophet

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.window import Window

from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import LinearRegression, RandomForestRegressor

In [3]:
SPARK_MASTER_IP = '172.18.0.2' 
spark = SparkSession.builder.appName("pyspark-taxi-forecasting_stage2") \
    .master(f"spark://{SPARK_MASTER_IP}:7077") \
    .config('spark.local.dir', 'spark_tmp/') \
    .config("spark.executor.cores", 2) \
    .config("spark.task.cpus", 2) \
    .getOrCreate()

In [4]:
spark

# Загрузим данные, создадим временные признаки

In [5]:
all_hours = spark.read.csv("all_hours.csv", header = True, inferSchema = True)

In [6]:
all_hours.show(5, False)
all_hours.printSchema()
all_hours.count()

+---+---------------------+-------------------+---------------+-----------+-----------+------------+--------------+---------------+
|_c0|pickup_community_area|hour_cons          |taxi_countdist3|trips_count|cost_median|miles_median|seconds_median|taxi_countdist8|
+---+---------------------+-------------------+---------------+-----------+-----------+------------+--------------+---------------+
|0  |0                    |2022-01-01 05:00:00|6              |7          |39.1       |3.24        |526.0         |6              |
|1  |0                    |2022-01-10 11:00:00|43             |46         |40.625     |11.375      |1349.0        |43             |
|2  |0                    |2022-01-16 14:00:00|29             |31         |47.4       |14.41       |1650.0        |29             |
|3  |0                    |2022-01-19 06:00:00|13             |14         |39.125     |9.515       |1620.5        |13             |
|4  |0                    |2022-02-16 06:00:00|15             |16         |5

1080144

In [7]:
column_list = ["pickup_community_area"]
  

Windowspec = Window.partitionBy(["pickup_community_area"]).orderBy(all_hours.hour_cons.desc())
  
all_hours_lagged = all_hours.filter(all_hours.hour_cons < '2023-07-31 23:00:00').withColumn(
    'med_cost_lagged1', lag(all_hours['cost_median'], -1).over(Windowspec)).withColumn(
    'med_miles_lagged1', lag(all_hours['miles_median'], -1).over(Windowspec)).withColumn(
    'med_seconds_lagged1', lag(all_hours['seconds_median'], -1).over(Windowspec)).withColumn(
    'trips_count_lagged1', lag(all_hours['trips_count'], -1).over(Windowspec)).withColumn(
    'trips_count_lagged2', lag(all_hours['trips_count'], -2).over(Windowspec)).withColumn(
    'trips_count_lagged3', lag(all_hours['trips_count'], -3).over(Windowspec)).withColumn(
    'trips_count_lagged12', lag(all_hours['trips_count'], -12).over(Windowspec)).withColumn(
    'trips_count_lagged24', lag(all_hours['trips_count'], -24).over(Windowspec)).withColumn(
    'trips_count_lagged_week', lag(all_hours['trips_count'], -24*7).over(Windowspec))

all_hours_lagged = all_hours_lagged.withColumn(
    'rolling_average_on3hours', avg(all_hours_lagged['trips_count_lagged1'],).over(Windowspec.rowsBetween(-3, -0))) \
    .withColumn('rolling_average_on24hours', avg(all_hours_lagged['trips_count_lagged1'],).over(Windowspec.rowsBetween(-24, -0)))

all_hours_lagged.filter('rolling_average_on3hours is NULL').show()

+---+---------------------+---------+---------------+-----------+-----------+------------+--------------+---------------+----------------+-----------------+-------------------+-------------------+-------------------+-------------------+--------------------+--------------------+-----------------------+------------------------+-------------------------+
|_c0|pickup_community_area|hour_cons|taxi_countdist3|trips_count|cost_median|miles_median|seconds_median|taxi_countdist8|med_cost_lagged1|med_miles_lagged1|med_seconds_lagged1|trips_count_lagged1|trips_count_lagged2|trips_count_lagged3|trips_count_lagged12|trips_count_lagged24|trips_count_lagged_week|rolling_average_on3hours|rolling_average_on24hours|
+---+---------------------+---------+---------------+-----------+-----------+------------+--------------+---------------+----------------+-----------------+-------------------+-------------------+-------------------+-------------------+--------------------+--------------------+--------------

In [8]:
all_hours_lagged = all_hours_lagged.na.drop('any').cache()

In [9]:
all_hours_lagged.count()

1066962

## Выделим валидационную выборку

In [10]:
train_df = all_hours_lagged.filter((all_hours_lagged.hour_cons < '2023-06-01 00:00:00')) 
train_df.show(5)
train_df.count()

+-------+---------------------+-------------------+---------------+-----------+-----------+------------+--------------+---------------+----------------+-----------------+-------------------+-------------------+-------------------+-------------------+--------------------+--------------------+-----------------------+------------------------+-------------------------+
|    _c0|pickup_community_area|          hour_cons|taxi_countdist3|trips_count|cost_median|miles_median|seconds_median|taxi_countdist8|med_cost_lagged1|med_miles_lagged1|med_seconds_lagged1|trips_count_lagged1|trips_count_lagged2|trips_count_lagged3|trips_count_lagged12|trips_count_lagged24|trips_count_lagged_week|rolling_average_on3hours|rolling_average_on24hours|
+-------+---------------------+-------------------+---------------+-----------+-----------+------------+--------------+---------------+----------------+-----------------+-------------------+-------------------+-------------------+-------------------+--------------

952848

In [11]:
valid_df = all_hours_lagged.filter(all_hours_lagged.hour_cons >= '2023-06-01 00:00:00')

# Рассмортим разные модели машинного обучения

## Проведем предсказания с помощью модели Prophet через Pandas_udf

In [12]:
train_proph = train_df.selectExpr( 'pickup_community_area',
    'hour_cons as ds', 'trips_count as y')
# Partition the data dfsp_partitionned
train_proph.createOrReplaceTempView("pickup_community_area")
sql = "select * from pickup_community_area"
train_proph = (spark.sql(sql)\
   .repartition(spark.sparkContext.defaultParallelism, 
   ['pickup_community_area'])).cache()
train_proph.explain()

== Physical Plan ==
InMemoryTableScan [pickup_community_area#18, ds#2024, y#2025]
   +- InMemoryRelation [pickup_community_area#18, ds#2024, y#2025], StorageLevel(disk, memory, deserialized, 1 replicas)
         +- Exchange hashpartitioning(pickup_community_area#18, 6), REPARTITION_BY_NUM, [plan_id=385]
            +- *(1) Project [pickup_community_area#18, hour_cons#19 AS ds#2024, trips_count#21 AS y#2025]
               +- *(1) Filter (isnotnull(hour_cons#19) AND (hour_cons#19 < 2023-06-01 00:00:00))
                  +- InMemoryTableScan [hour_cons#19, pickup_community_area#18, trips_count#21], [isnotnull(hour_cons#19), (hour_cons#19 < 2023-06-01 00:00:00)]
                        +- InMemoryRelation [_c0#17, pickup_community_area#18, hour_cons#19, taxi_countdist3#20, trips_count#21, cost_median#22, miles_median#23, seconds_median#24, taxi_countdist8#25, med_cost_lagged1#96, med_miles_lagged1#107, med_seconds_lagged1#119, trips_count_lagged1#132, trips_count_lagged2#146, trips_count

In [13]:
valid_proph = all_hours_lagged.filter(all_hours_lagged.hour_cons >= '2023-06-01 00:00:00')
valid_proph = valid_proph.selectExpr( 'pickup_community_area',
    'hour_cons as ds', 'trips_count as y')
valid_proph = valid_proph.withColumn("y",col("y").cast(DoubleType()))
valid_proph.sort('ds', 'pickup_community_area').show(5)

valid_proph.count()

+---------------------+-------------------+----+
|pickup_community_area|                 ds|   y|
+---------------------+-------------------+----+
|                    0|2023-06-01 00:00:00|22.0|
|                    1|2023-06-01 00:00:00| 0.0|
|                    2|2023-06-01 00:00:00| 0.0|
|                    3|2023-06-01 00:00:00| 5.0|
|                    4|2023-06-01 00:00:00| 1.0|
+---------------------+-------------------+----+
only showing top 5 rows



114114

In [14]:
# Define a schema
schema = StructType([ \
                     StructField('pickup_community_area', IntegerType()), 
                     StructField('ds', TimestampType()),
                     StructField('y', FloatType()),
                     StructField('yhat', DoubleType()),
                     StructField('daily', DoubleType()),
                     StructField('weekly', DoubleType())
                    ])

In [15]:
# define the Pandas UDF
@pandas_udf(schema, PandasUDFType.GROUPED_MAP)
def apply_model(store_pd):  # instantiate the model and set parameters
  model = Prophet(
      interval_width=0.95,
      growth='linear',
      n_changepoints = 150,
      daily_seasonality=True,
      weekly_seasonality=True,
      yearly_seasonality=True,
      seasonality_mode='additive'
  )  # fit the model to historical data
  model.fit(store_pd)  # Create a data frame that lists 90 dates starting from Jan 1 2018
  future = model.make_future_dataframe(
      periods=1465, #valid_proph.filter(valid_proph.pickup_community_area==community_num).count(),
      freq='h',
      include_history=True
   )  # Out of sample prediction
  prediction = model.predict(future)  # Create a data frame that contains store, item, y, and yhat
  f_pd = prediction[['ds', 'yhat', 'daily', 'weekly']]
  st_pd = store_pd[['ds', 'pickup_community_area', 'y']]
  result_pd = f_pd.join(st_pd.set_index('ds'), on='ds', how='left')  # fill store and item
  result_pd['pickup_community_area'] = store_pd['pickup_community_area'].iloc[0]
  return result_pd[['pickup_community_area', 'ds', 'y', 'yhat',
                    'daily', 'weekly']]# Apply the function to all store-items
# Print the results - calculate the time to run
results = train_proph.groupby(['pickup_community_area']).apply(apply_model).cache()

results.show()

/usr/local/spark/python/pyspark/sql/pandas/group_ops.py:103: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


+---------------------+-------------------+----+-------------------+--------------------+--------------------+
|pickup_community_area|                 ds|   y|               yhat|               daily|              weekly|
+---------------------+-------------------+----+-------------------+--------------------+--------------------+
|                    2|2022-01-08 00:00:00| 2.0| 1.8385607045261043| -1.5888611750253627|-0.08040349901784355|
|                    2|2022-01-08 01:00:00| 2.0| 2.4067981636683955| -1.0670945633086735|-0.04442358518432663|
|                    2|2022-01-08 02:00:00| 0.0| 2.8665364039663244| -0.6522147770712824|-0.01005577246055...|
|                    2|2022-01-08 03:00:00| 0.0|  3.000428355931298| -0.5610094280911881|0.022140806809180104|
|                    2|2022-01-08 04:00:00| 2.0| 3.0714225987220933|  -0.529999553779111|0.051636094866233904|
|                    2|2022-01-08 05:00:00| 2.0| 3.3589709146202287|-0.27923991313226026|  0.0779369589563707|
|

In [16]:
results[['yhat']].count()

1067118

## Оценим полученный прогноз

In [17]:
predictions = results.filter(results['ds'] >= '2023-06-01 00:00:00').select(
    'pickup_community_area', 'ds', 'yhat')
predictions = predictions.withColumn("yhat_int",F.round(predictions["yhat"],0))
predictions = predictions.withColumn('yhat_int', F.when((F.col("yhat_int") <= 0), 0)\
    .otherwise(predictions.yhat_int)).cache()
    

predictions.filter('pickup_community_area == 76').show(5)
predictions.count()

+---------------------+-------------------+-------------------+--------+
|pickup_community_area|                 ds|               yhat|yhat_int|
+---------------------+-------------------+-------------------+--------+
|                   76|2023-06-01 00:00:00|-15.311291612860053|     0.0|
|                   76|2023-06-01 01:00:00| 19.506413591095033|    20.0|
|                   76|2023-06-01 02:00:00|  71.10845290355063|    71.0|
|                   76|2023-06-01 03:00:00|  95.46551987383731|    95.0|
|                   76|2023-06-01 04:00:00| 100.86699334545446|   101.0|
+---------------------+-------------------+-------------------+--------+
only showing top 5 rows



114270

In [18]:
valid_df_withyhat = valid_proph.join(predictions, on=['ds','pickup_community_area'] , how='inner')  # fill store and item

In [19]:
valid_df_withyhat.show(5)
valid_df_withyhat.count()

+-------------------+---------------------+---+------------------+--------+
|                 ds|pickup_community_area|  y|              yhat|yhat_int|
+-------------------+---------------------+---+------------------+--------+
|2023-07-31 22:00:00|                   31|0.0|0.7512249903666159|     1.0|
|2023-07-31 21:00:00|                   31|0.0|0.9469810492014389|     1.0|
|2023-07-31 20:00:00|                   31|0.0|1.4176049221359113|     1.0|
|2023-07-31 19:00:00|                   31|0.0|1.8156671439467194|     2.0|
|2023-07-31 18:00:00|                   31|1.0|1.8163198173088635|     2.0|
+-------------------+---------------------+---+------------------+--------+
only showing top 5 rows



114114

In [20]:
evaluator = RegressionEvaluator(predictionCol="yhat_int", labelCol='y', metricName='mae')
print("Prphet MAE: {0}".format(evaluator.evaluate(valid_df_withyhat)))

Prphet MAE: 8.231461520935206


In [21]:
print('MAPE:',
    valid_df_withyhat.select(avg((100*abs((valid_df_withyhat.y - valid_df_withyhat.yhat_int) / valid_df_withyhat.y)))).collect())

MAPE: [Row(avg((abs(((y - yhat_int) / y)) * 100))=93.46754521142931)]


    мае - 7,9. Предсказания модели Prоphet показали низкое качество.
    Мы будем обучать другие модели, из Prophet возьмем данные о сезонности.

In [22]:
results.columns

['pickup_community_area', 'ds', 'y', 'yhat', 'daily', 'weekly']

## Проведем обучение линейной регрессии

### Проведем стандартизацию и соберем все признаки в 1 вектор

In [23]:
train_lr = all_hours_lagged.filter((all_hours_lagged.hour_cons < '2023-06-01 00:00:00')) \
    .selectExpr( 'pickup_community_area',
    'med_cost_lagged1','med_miles_lagged1','med_seconds_lagged1',
    'trips_count_lagged1', 'trips_count_lagged2', 'trips_count_lagged3', 
    'trips_count_lagged12', 'trips_count_lagged24','trips_count_lagged_week',
    'rolling_average_on3hours',
    'rolling_average_on24hours',
    'hour_cons as ds', 'trips_count as y') \
        .join(results['pickup_community_area', 'ds', 'daily', 'weekly'], on=['ds','pickup_community_area'] , how='inner').cache()

In [24]:
train_lr.printSchema()
train_lr.show(5)
train_lr.count()

root
 |-- ds: timestamp (nullable = true)
 |-- pickup_community_area: integer (nullable = true)
 |-- med_cost_lagged1: double (nullable = true)
 |-- med_miles_lagged1: double (nullable = true)
 |-- med_seconds_lagged1: double (nullable = true)
 |-- trips_count_lagged1: integer (nullable = true)
 |-- trips_count_lagged2: integer (nullable = true)
 |-- trips_count_lagged3: integer (nullable = true)
 |-- trips_count_lagged12: integer (nullable = true)
 |-- trips_count_lagged24: integer (nullable = true)
 |-- trips_count_lagged_week: integer (nullable = true)
 |-- rolling_average_on3hours: double (nullable = true)
 |-- rolling_average_on24hours: double (nullable = true)
 |-- y: integer (nullable = true)
 |-- daily: double (nullable = true)
 |-- weekly: double (nullable = true)

+-------------------+---------------------+----------------+-----------------+-------------------+-------------------+-------------------+-------------------+--------------------+--------------------+---------------

952848

In [25]:
featureCols = [
    'med_cost_lagged1','med_miles_lagged1','med_seconds_lagged1',
    'trips_count_lagged1', 'trips_count_lagged2', 'trips_count_lagged3', 
    'trips_count_lagged12', 'trips_count_lagged24','trips_count_lagged_week',
    'rolling_average_on3hours',
    'rolling_average_on24hours',
     'daily', 'weekly'
]

In [26]:
# положить фичи в вектор
assembler = VectorAssembler(inputCols=featureCols, outputCol="features") 
assembled_df = assembler.transform(train_lr)

In [27]:
assembled_df.show(20, False)

+-------------------+---------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+--------------------+--------------------+-----------------------+------------------------+-------------------------+---+---------------------+---------------------+--------------------------------------------------------------------------------------------------------------+
|ds                 |pickup_community_area|med_cost_lagged1  |med_miles_lagged1 |med_seconds_lagged1|trips_count_lagged1|trips_count_lagged2|trips_count_lagged3|trips_count_lagged12|trips_count_lagged24|trips_count_lagged_week|rolling_average_on3hours|rolling_average_on24hours|y  |daily                |weekly               |features                                                                                                      |
+-------------------+---------------------+------------------+------------------+-------------------+-------------------+---

In [28]:
standardScaler = StandardScaler(inputCol="features", outputCol="features_scaled", withStd = True)

In [29]:
train_lr_scaled = standardScaler.fit(assembled_df).transform(assembled_df).cache()

In [30]:
train_lr_scaled.show(5)
train_lr_scaled.count()

+-------------------+---------------------+----------------+-----------------+-------------------+-------------------+-------------------+-------------------+--------------------+--------------------+-----------------------+------------------------+-------------------------+---+--------------------+--------------------+--------------------+--------------------+
|                 ds|pickup_community_area|med_cost_lagged1|med_miles_lagged1|med_seconds_lagged1|trips_count_lagged1|trips_count_lagged2|trips_count_lagged3|trips_count_lagged12|trips_count_lagged24|trips_count_lagged_week|rolling_average_on3hours|rolling_average_on24hours|  y|               daily|              weekly|            features|     features_scaled|
+-------------------+---------------------+----------------+-----------------+-------------------+-------------------+-------------------+-------------------+--------------------+--------------------+-----------------------+------------------------+-----------------------

952848

### Проведем стандартизацию валидационных признаков

In [31]:
valid_lr = all_hours_lagged.filter(all_hours_lagged.hour_cons >= '2023-06-01 00:00:00') \
    .selectExpr('pickup_community_area',
    'med_cost_lagged1','med_miles_lagged1','med_seconds_lagged1',
    'trips_count_lagged1', 'trips_count_lagged2', 'trips_count_lagged3', 
    'trips_count_lagged12', 'trips_count_lagged24','trips_count_lagged_week',
    'rolling_average_on3hours',
    'rolling_average_on24hours',
    'hour_cons as ds', 'trips_count as y') \
        .join(results['pickup_community_area', 'ds', 'daily', 'weekly'], on=['ds','pickup_community_area'] , how='inner').cache()
valid_lr.count()

114114

In [32]:
valid_lr_scaled = assembler.transform(valid_lr)

In [33]:
valid_lr_scaled = standardScaler.fit(train_lr_scaled[['features']]).transform(valid_lr_scaled)

In [34]:
valid_lr_scaled.cache()
valid_lr_scaled.show(5, False)
valid_lr_scaled.count()

+-------------------+---------------------+----------------+------------------+-------------------+-------------------+-------------------+-------------------+--------------------+--------------------+-----------------------+------------------------+-------------------------+---+-------------------+--------------------+-----------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|ds                 |pickup_community_area|med_cost_lagged1|med_miles_lagged1 |med_seconds_lagged1|trips_count_lagged1|trips_count_lagged2|trips_count_lagged3|trips_count_lagged12|trips_count_lagged24|trips_count_lagged_week|rolling_average_on3hours|rolling_average_on24hours|y  |daily              |weekly   

114114

In [35]:
train_data = train_lr_scaled.selectExpr('y',
    'features_scaled')

In [36]:
lr = (LinearRegression(featuresCol='features_scaled', labelCol="y", predictionCol='y_pred')) 
#                               maxIter=10, regParam=0.3, elasticNetParam=0.8, standardization=False))

In [37]:
linearModel = lr.fit(train_data)

In [38]:
coeff_df = pd.DataFrame({"Feature": ["Intercept"] + featureCols, "Co-efficients": np.insert(linearModel.coefficients.toArray(), 0, linearModel.intercept)})
coeff_df = coeff_df[["Feature", "Co-efficients"]]
display(coeff_df)

,Feature,Co-efficients
0,Intercept,0.099739
1,med_cost_lagged1,-0.073153
2,med_miles_lagged1,-0.124021
3,med_seconds_lagged1,-0.017947
4,trips_count_lagged1,9.325066
5,trips_count_lagged2,-3.536604
6,trips_count_lagged3,1.887217
7,trips_count_lagged12,-0.068058
8,trips_count_lagged24,1.759492
9,trips_count_lagged_week,2.425349


In [39]:
valid_data = valid_lr_scaled.selectExpr('y',
    'features_scaled')

In [40]:
# Прогнозы
predictions = linearModel.transform(valid_data)

In [41]:
# Вытаскиваем предсказания и истинные ответы)
predandlabels = predictions.select("y_pred", "y").withColumn("y_pred",F.round(predictions["y_pred"],0)) \
                                                        #.withColumn("y", F.when((F.col("y") <= 0), 0.0001))
predandlabels = predandlabels.withColumn('y_pred', F.when((F.col("y_pred") <= 0), 0)\
    .otherwise(predandlabels.y_pred)).cache()
predandlabels.show()
predandlabels.count()

+------+---+
|y_pred|  y|
+------+---+
|  30.0| 29|
|   1.0|  0|
|   3.0|  2|
|   2.0|  2|
|   1.0|  1|
|   9.0|  6|
| 275.0|261|
|  52.0| 62|
|   9.0| 17|
|   0.0|  0|
|   0.0|  0|
|   2.0|  3|
|   1.0|  1|
|   9.0|  9|
|   1.0|  1|
|   4.0|  1|
| 135.0|160|
|  52.0| 47|
|   0.0|  0|
|   0.0|  0|
+------+---+
only showing top 20 rows



114114

In [42]:
evaluator = RegressionEvaluator(predictionCol="y_pred", labelCol='y', metricName='mae')
print("LinearRegression MAE: {0}".format(evaluator.evaluate(predandlabels)))

LinearRegression MAE: 1.680433601486233


In [43]:
print('MAPE:',
    predandlabels.select(avg((100*abs((predandlabels.y - predandlabels.y_pred) / predandlabels.y)))).collect())

MAPE: [Row(avg((abs(((y - y_pred) / y)) * 100))=43.281272503358764)]


    на дополнительных сезонностях
    МАЕ: 1.680433601486233
    МАРЕ: 43.281272503358764

In [44]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[assembler, standardScaler, lr])
piped_lr = pipeline.fit(train_lr)
piped_prediction = []
piped_prediction = piped_lr.transform(valid_lr)
piped_prediction = piped_prediction.select("y_pred", "y").withColumn("y_pred",F.round(piped_prediction["y_pred"],0)) 
                                                        #.withColumn("y", F.when((F.col("y") <= 0), 0.0001))
piped_prediction = piped_prediction.withColumn('y_pred', F.when((F.col("y_pred") <= 0), 0)\
    .otherwise(piped_prediction.y_pred)).cache()
print("LinearRegression MAE: {0}".format(evaluator.evaluate(piped_prediction)))
print('MAPE:',
    piped_prediction.select(avg((100*abs((
        piped_prediction.y - piped_prediction.y_pred) / piped_prediction.y)))).collect())
piped_prediction.show(10)
print(piped_prediction.count())

LinearRegression MAE: 1.680433601486233
MAPE: [Row(avg((abs(((y - y_pred) / y)) * 100))=43.281272503358764)]
+------+---+
|y_pred|  y|
+------+---+
|  30.0| 29|
|   1.0|  0|
|   3.0|  2|
|   2.0|  2|
|   1.0|  1|
|   9.0|  6|
| 275.0|261|
|  52.0| 62|
|   9.0| 17|
|   0.0|  0|
+------+---+
only showing top 10 rows

114114


## Обучим лес случайных деревьев

In [45]:
%%time

rf = RandomForestRegressor(labelCol="y", featuresCol="features_scaled", predictionCol='y_pred',
                                  numTrees = 10, maxDepth = 10) \

rfmodel = rf.fit(train_data)
rfpredictions = rfmodel.transform(valid_data)
rfpredictions.show()
print("RandomForest MAE: {0}".format(evaluator.evaluate(rfpredictions)))

+---+--------------------+--------------------+
|  y|     features_scaled|              y_pred|
+---+--------------------+--------------------+
| 29|[1.97822447731942...|  28.579687975662573|
|  0|[1.03172705166319...|  0.5272077483298967|
|  2|[0.57754286044189...|  3.6034865359503376|
|  2|[1.31881631821295...|  1.7087051117407053|
|  1|[1.35694536142659...|  0.6708635138799587|
|  6|[0.72804043689102...|   8.212283171451308|
|261|[0.52124644957940...|  255.44962131868883|
| 62|[0.71772316637439...|   49.40804531016315|
| 17|[1.73599290866806...|  11.958775464645573|
|  0|(13,[5,6,9,10,11,...|  0.2610302572056099|
|  0|(13,[7,8,10,11,12...| 0.10692136380656592|
|  3|(13,[5,6,8,9,10,1...|  1.8023644752023202|
|  1|(13,[7,8,9,10,11,...|  0.6298199713389501|
|  9|[0.98686935376479...|  12.064700919774513|
|  1|[0.0,0.0,0.0,0.0,...|  2.0211820637573217|
|  1|[0.89715395796799...|  2.1960123200373918|
|160|[0.54950679925539...|  116.14850927909065|
| 47|[2.13522641996382...|  44.472467777

    Лес деревьев потребляет много ресурсов и не дает высоких результатов. Кросс-валидация не проходит из-за высокого потребления ресурсов.
    МАЕ случайного леса 2.6110337641144907
    В дальнейшем будем использовать модели линейной регрессии

## Выбор модели по результатам валидации:
    Для предсказания количества поездок в следующий час по каждому району была выбрана модель линейной регрессии.
    Окончательные ошибки отдельных Линейных регрессий для каждого района составили    
МАЕ: 1.680433601486233

МАРЕ: 43.281272503358764

# Реализуем выбранную модель для каждого региона через функцию

In [46]:
def modelsandpredictions(func_community):
        
    temp_train = train_lr.filter(train_lr.pickup_community_area == func_community)
    temp_valid = valid_lr.filter(valid_lr.pickup_community_area == func_community)
    
    temp_lr = pipeline.fit(temp_train)
    temp_pred = temp_lr.transform(temp_valid)
    temp_predandtrue = temp_pred.select(
                                        'pickup_community_area',
                                        'ds',
                                        "y_pred", 
                                        "y"
                                        ).withColumn("y_pred",F.round(temp_pred["y_pred"],0)) 
    temp_predandtrue = temp_predandtrue.withColumn('y_pred', F.when((F.col("y_pred") <= 0), 0)\
        .otherwise(temp_predandtrue.y_pred)).cache()
    #temp_lr = lr.fit(temp_train.union(temp_valid))
    #temp_lr.write().overwrite().save("/models/lr{0}".format(i))
    return temp_predandtrue

In [47]:
preds_schema = StructType([ \
    StructField("pickup_community_area",IntegerType (),True), \
    StructField("ds",TimestampType(),True), \
    StructField("y_pred",FloatType(),True), \
    StructField("y", FloatType(), True), \

  ])

In [48]:
%%time
valid_preds = spark.createDataFrame([],preds_schema)
for i in range(78):
    valid_preds = valid_preds.union(modelsandpredictions(i))
valid_preds = valid_preds.cache()

CPU times: user 6.04 s, sys: 2.69 s, total: 8.73 s
Wall time: 5min 19s


In [49]:
valid_preds.show(5)
valid_preds.count()

+---------------------+-------------------+------+----+
|pickup_community_area|                 ds|y_pred|   y|
+---------------------+-------------------+------+----+
|                    0|2023-06-02 01:00:00|   9.0|17.0|
|                    0|2023-06-04 06:00:00|   8.0| 6.0|
|                    0|2023-06-05 18:00:00|  38.0|37.0|
|                    0|2023-06-13 12:00:00|  41.0|37.0|
|                    0|2023-06-16 21:00:00|  23.0|18.0|
+---------------------+-------------------+------+----+
only showing top 5 rows



114114

In [50]:
%%time
print("LinearRegression MAE: {0}".format(evaluator.evaluate(valid_preds)))

LinearRegression MAE: 1.5661268556005399
CPU times: user 1.5 s, sys: 725 ms, total: 2.23 s
Wall time: 5min 42s


In [51]:
%%time
print('MAPE:',
    valid_preds.select(avg((100*abs((valid_preds.y - valid_preds.y_pred) / valid_preds.y)))).collect())

MAPE: [Row(avg((abs(((y - y_pred) / y)) * 100))=38.60706497578098)]
CPU times: user 1.63 s, sys: 658 ms, total: 2.29 s
Wall time: 5min 43s


    Обучение отдельных моделей для каждого района свою показали эффективность уменьшив МАЕ на 0.114

# Проверим на тестовых данных

In [52]:
spark.stop()

In [53]:
SPARK_MASTER_IP = '172.18.0.2' 
spark = SparkSession.builder.appName("pyspark-taxi-forecasting_stage2") \
    .master(f"spark://{SPARK_MASTER_IP}:7077") \
    .config('spark.local.dir', 'spark_tmp/') \
    .config("spark.executor.cores", 2) \
    .config("spark.task.cpus", 2) \
    .getOrCreate()

In [55]:
all_hours = spark.read.csv("all_hours.csv", header = True, inferSchema = True)

In [56]:
test = spark.read.csv("y_true_2023-07-31_23-00_UTC0.csv", header = True, inferSchema = True)
test = test.withColumn('ds', test['hours']) \
    .withColumn('pickup_community_area', test['Pickup Community Area']) \
    .withColumn('y', test['trips_count'])

In [57]:
test.show()
test.count()

+---------------------+-------------------+-----------+-------------------+---------------------+---+
|Pickup Community Area|              hours|trips_count|                 ds|pickup_community_area|  y|
+---------------------+-------------------+-----------+-------------------+---------------------+---+
|                    0|2023-07-31 23:00:00|         22|2023-07-31 23:00:00|                    0| 22|
|                    1|2023-07-31 23:00:00|          2|2023-07-31 23:00:00|                    1|  2|
|                    2|2023-07-31 23:00:00|          1|2023-07-31 23:00:00|                    2|  1|
|                    3|2023-07-31 23:00:00|          5|2023-07-31 23:00:00|                    3|  5|
|                    4|2023-07-31 23:00:00|          0|2023-07-31 23:00:00|                    4|  0|
|                    5|2023-07-31 23:00:00|          4|2023-07-31 23:00:00|                    5|  4|
|                    6|2023-07-31 23:00:00|         49|2023-07-31 23:00:00|       

78

In [58]:
column_list = ["pickup_community_area"]
  

Windowspec = Window.partitionBy(["pickup_community_area"]).orderBy(all_hours.hour_cons.desc())
  
all_hours_lagged = all_hours.withColumn(
    'med_cost_lagged1', lag(all_hours['cost_median'], -1).over(Windowspec)).withColumn(
    'med_miles_lagged1', lag(all_hours['miles_median'], -1).over(Windowspec)).withColumn(
    'med_seconds_lagged1', lag(all_hours['seconds_median'], -1).over(Windowspec)).withColumn(
    'trips_count_lagged1', lag(all_hours['trips_count'], -1).over(Windowspec)).withColumn(
    'trips_count_lagged2', lag(all_hours['trips_count'], -2).over(Windowspec)).withColumn(
    'trips_count_lagged3', lag(all_hours['trips_count'], -3).over(Windowspec)).withColumn(
    'trips_count_lagged12', lag(all_hours['trips_count'], -12).over(Windowspec)).withColumn(
    'trips_count_lagged24', lag(all_hours['trips_count'], -24).over(Windowspec)).withColumn(
    'trips_count_lagged_week', lag(all_hours['trips_count'], -24*7).over(Windowspec))

all_hours_lagged = all_hours_lagged.withColumn(
    'rolling_average_on3hours', avg(all_hours_lagged['trips_count_lagged1'],).over(Windowspec.rowsBetween(-3, -0))) \
    .withColumn('rolling_average_on24hours', avg(all_hours_lagged['trips_count_lagged1'],).over(Windowspec.rowsBetween(-24, -0))) \
    .na.drop('any').cache()


## Проведем предсказания с помощью Prophet через Pandas_udf для выделения сезонностей

In [59]:
train_proph = all_hours_lagged.filter((all_hours_lagged.hour_cons < '2023-07-31 23:00:00')) \
    .selectExpr( 'pickup_community_area',
    'hour_cons as ds', 'trips_count as y')
# Partition the data dfsp_partitionned
train_proph.createOrReplaceTempView("pickup_community_area")
sql = "select * from pickup_community_area"
train_proph = (spark.sql(sql)\
   .repartition(spark.sparkContext.defaultParallelism, 
   ['pickup_community_area'])).cache()
train_proph.explain()

== Physical Plan ==
InMemoryTableScan [pickup_community_area#188690, ds#189087, y#189088]
   +- InMemoryRelation [pickup_community_area#188690, ds#189087, y#189088], StorageLevel(disk, memory, deserialized, 1 replicas)
         +- Exchange hashpartitioning(pickup_community_area#188690, 6), REPARTITION_BY_NUM, [plan_id=22167]
            +- *(1) Project [pickup_community_area#188690, hour_cons#188691 AS ds#189087, trips_count#188693 AS y#189088]
               +- *(1) Filter (isnotnull(hour_cons#188691) AND (hour_cons#188691 < 2023-07-31 23:00:00))
                  +- InMemoryTableScan [hour_cons#188691, pickup_community_area#188690, trips_count#188693], [isnotnull(hour_cons#188691), (hour_cons#188691 < 2023-07-31 23:00:00)]
                        +- InMemoryRelation [_c0#188689, pickup_community_area#188690, hour_cons#188691, taxi_countdist3#188692, trips_count#188693, cost_median#188694, miles_median#188695, seconds_median#188696, taxi_countdist8#188697, med_cost_lagged1#188788, med

In [60]:
valid_proph = all_hours_lagged.filter(all_hours_lagged.hour_cons == '2023-07-31 23:00:00') \
    .selectExpr( 'pickup_community_area',
    'hour_cons as ds', 'trips_count as y') \
    .withColumn("y",col("y").cast(DoubleType()))

In [61]:
# Define a schema
schema = StructType([ \
                     StructField('pickup_community_area', IntegerType()), 
                     StructField('ds', TimestampType()),
                     StructField('y', FloatType()),
                     StructField('yhat', DoubleType()),
                     StructField('daily', DoubleType()),
                     StructField('weekly', DoubleType())
                    ])

In [62]:
# define the Pandas UDF
@pandas_udf(schema, PandasUDFType.GROUPED_MAP)
def apply_model(store_pd):  # instantiate the model and set parameters
  model = Prophet(
      interval_width=0.95,
      growth='linear',
      n_changepoints = 150,
      daily_seasonality=True,
      weekly_seasonality=True,
      yearly_seasonality=True,
      seasonality_mode='additive'
  )  # fit the model to historical data
  model.fit(store_pd)  # Create a data frame that lists 90 dates starting from Jan 1 2018
  future = model.make_future_dataframe(
      periods=1, #valid_proph.filter(valid_proph.pickup_community_area==community_num).count(),
      freq='h',
      include_history=True
   )  # Out of sample prediction
  prediction = model.predict(future)  # Create a data frame that contains store, item, y, and yhat
  f_pd = prediction[['ds', 'yhat', 'daily', 'weekly']]
  st_pd = store_pd[['ds', 'pickup_community_area', 'y']]
  result_pd = f_pd.join(st_pd.set_index('ds'), on='ds', how='left')  # fill store and item
  result_pd['pickup_community_area'] = store_pd['pickup_community_area'].iloc[0]
  return result_pd[['pickup_community_area', 'ds', 'y', 'yhat',
                    'daily', 'weekly']]# Apply the function to all store-items
# Print the results - calculate the time to run
results = train_proph.groupby(['pickup_community_area']).apply(apply_model).cache()


/usr/local/spark/python/pyspark/sql/pandas/group_ops.py:103: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


## Переопределим тренировочную и тестовую выборки

In [63]:
train_lr = all_hours_lagged.filter((all_hours_lagged.hour_cons < '2023-07-31 23:00:00')) \
    .selectExpr( 'pickup_community_area',
    'med_cost_lagged1','med_miles_lagged1','med_seconds_lagged1',
    'trips_count_lagged1', 'trips_count_lagged2', 'trips_count_lagged3', 
    'trips_count_lagged12', 'trips_count_lagged24','trips_count_lagged_week',
    'rolling_average_on3hours',
    'rolling_average_on24hours',
    'hour_cons as ds', 'trips_count as y') \
        .join(results['pickup_community_area', 'ds', 'daily', 'weekly'], on=['ds','pickup_community_area'] , how='inner').cache()

In [64]:
test_lr = all_hours_lagged.filter(all_hours_lagged.hour_cons == '2023-07-31 23:00:00') \
    .selectExpr('pickup_community_area',
    'med_cost_lagged1','med_miles_lagged1','med_seconds_lagged1',
    'trips_count_lagged1', 'trips_count_lagged2', 'trips_count_lagged3', 
    'trips_count_lagged12', 'trips_count_lagged24','trips_count_lagged_week',
    'rolling_average_on3hours',
    'rolling_average_on24hours',
    'hour_cons as ds').join(     #'trips_count as y') \
    results['pickup_community_area', 'ds', 'daily', 'weekly'], on=['ds','pickup_community_area'] , how='inner').cache()

In [65]:
featureCols = [
    'med_cost_lagged1','med_miles_lagged1','med_seconds_lagged1',
    'trips_count_lagged1', 'trips_count_lagged2', 'trips_count_lagged3', 
    'trips_count_lagged12', 'trips_count_lagged24','trips_count_lagged_week',
    'rolling_average_on3hours',
    'rolling_average_on24hours',
     'daily', 'weekly'
]

## Создадим pipeline

In [66]:
# положить фичи в вектор
assembler = VectorAssembler(inputCols=featureCols, outputCol="features") 

In [67]:
standardScaler = StandardScaler(inputCol="features", outputCol="features_scaled", withStd = True)

In [68]:
lr = (LinearRegression(featuresCol='features_scaled', labelCol="y", predictionCol='y_pred')) 
#                               maxIter=10, regParam=0.3, elasticNetParam=0.8, standardization=False))

In [69]:
pipeline = Pipeline(stages=[assembler, standardScaler, lr])

In [70]:
evaluator = RegressionEvaluator(predictionCol="y_pred", labelCol='y', metricName='mae')

## проведем предсказания для челевого часа аналогично валидационным данным

In [71]:
def modelsandpredictions(func_community):
        
    temp_train = train_lr.filter(train_lr.pickup_community_area == func_community)
    temp_test = test_lr.filter(test_lr.pickup_community_area == func_community)
    
    temp_lr = pipeline.fit(temp_train)
    temp_pred = temp_lr.transform(temp_test)
    temp_predandtrue = temp_pred.select(
                                        'pickup_community_area',
                                        'ds',
                                        "y_pred", 
                                        ).withColumn("y_pred",F.round(temp_pred["y_pred"],0)) 
    temp_predandtrue = temp_predandtrue.withColumn('y_pred', F.when((F.col("y_pred") <= 0), 0)\
        .otherwise(temp_predandtrue.y_pred)).cache()
    #temp_lr = lr.fit(temp_train.union(temp_test))
    #temp_lr.write().overwrite().save("/models/lr{0}".format(i))
    return temp_predandtrue

In [72]:
preds_schema = StructType([ \
    StructField("pickup_community_area",IntegerType (),True), \
    StructField("ds",TimestampType(),True), \
    StructField("y_pred",FloatType(),True) \
  ])

In [73]:
%%time
test_preds = spark.createDataFrame([],preds_schema)
for i in range(78):
    test_preds = test_preds.union(modelsandpredictions(i))

CPU times: user 6.51 s, sys: 2.84 s, total: 9.34 s
Wall time: 15min 3s


In [74]:
test_preds.show(5)
test_preds.count()

+---------------------+-------------------+------+
|pickup_community_area|                 ds|y_pred|
+---------------------+-------------------+------+
|                    0|2023-07-31 23:00:00|  30.0|
|                    1|2023-07-31 23:00:00|   2.0|
|                    2|2023-07-31 23:00:00|   0.0|
|                    3|2023-07-31 23:00:00|   3.0|
|                    4|2023-07-31 23:00:00|   2.0|
+---------------------+-------------------+------+
only showing top 5 rows



78

In [75]:
trueandpreds = test.join(test_preds, on=['ds','pickup_community_area'], how='inner') \
    .select(
        test.ds,
        test.pickup_community_area,
        test.y,
        test_preds.y_pred).cache()

## Посмотрим финальные предсказания и оценим их

In [76]:
%%time
trueandpreds.sort(trueandpreds.pickup_community_area.desc()).show(80)
print("LinearRegression MAE: {0}".format(evaluator.evaluate(trueandpreds)))
print('MAPE:',
    trueandpreds.select(avg((100*abs((trueandpreds.y - trueandpreds.y_pred) / trueandpreds.y)))).collect())

+-------------------+---------------------+---+------+
|                 ds|pickup_community_area|  y|y_pred|
+-------------------+---------------------+---+------+
|2023-07-31 23:00:00|                   77|  2|   4.0|
|2023-07-31 23:00:00|                   76|160| 205.0|
|2023-07-31 23:00:00|                   75|  0|   3.0|
|2023-07-31 23:00:00|                   74|  0|   0.0|
|2023-07-31 23:00:00|                   73|  0|   0.0|
|2023-07-31 23:00:00|                   72|  0|   1.0|
|2023-07-31 23:00:00|                   71|  0|   2.0|
|2023-07-31 23:00:00|                   70|  0|   2.0|
|2023-07-31 23:00:00|                   69|  2|   0.0|
|2023-07-31 23:00:00|                   68|  0|   3.0|
|2023-07-31 23:00:00|                   67|  0|   1.0|
|2023-07-31 23:00:00|                   66|  2|   1.0|
|2023-07-31 23:00:00|                   65|  0|   1.0|
|2023-07-31 23:00:00|                   64|  0|   0.0|
|2023-07-31 23:00:00|                   63|  0|   0.0|
|2023-07-3

# Итог:
    Для предсказания количества заказов такси в каждом районе Чикаго в следующий час предлагается использовать модели машинного обучения на основе Линейных регрессий, отдельно обученные на данных для этого региона.
    Их окончательные ошибки составили
    МАЕ: 1.9102564102564104
    МАРЕ: 63.55259106922181

    Возможно, тестовые данные были обработаны иначе. Необходимо определить и скоординировать в частности, работу с пропусками и заполнение района 0 в тестовых и тренировочных данных.